<a href="https://colab.research.google.com/github/Anil951/Early-detection-of-mental-health/blob/main/implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("-----------------EARLY DETECTION OF MENTAL HEALTH ISSUES IN ADOLESCENTS SYSTEM------------------------")

print("\n\nWE ARE DEALING IN 3 STAGES\n\n")

print("in STAGE-1: All the social media platform data of the user is collected and analysed with our trained model- ")
print("to get NORMAL/NOT NORMAL in corresponding to MENTAL HEALTH - a score will be assigned\n")
print("in STAGE-2: The use shoudl give us his/her recent exam scores and remarks given by the teachers/mentors - ")
print("our trained model will evaluate and assign a score\n")
print("in STAGE-3: The use can submit his health record scans ")

print("\n\n------------IN FINAL WE GIVE ANY RECOMMENDATIONS BASED ON OUR RESULTS------------")


-----------------EARLY DETECTION OF MENTAL HEALTH ISSUES IN ADOLESCENTS SYSTEM------------------------


WE ARE DEALING IN 3 STAGES


in STAGE-1: All the social media platform data of the user is collected and analysed with our trained model- 
to get NORMAL/NOT NORMAL in corresponding to MENTAL HEALTH - a score will be assigned

in STAGE-2: The use shoudl give us his/her recent exam scores and remarks given by the teachers/mentors - 
our trained model will evaluate and assign a score

in STAGE-3: The use can submit his health record scans 


------------IN FINAL WE GIVE ANY RECOMMENDATIONS BASED ON OUR RESULTS------------


In [ ]:
# STAGE 1

# COLLECTING USER SOCIAL MEDIA DATA (for prototype purpose we are using REDDIT AND WHATSAPP CHATS)


In [ ]:
# COLLECTING USER INTERACTION DATA FROM REDDIT (comments the user has done & the hashtags he/she has been using)

In [3]:
# pip install asyncpraw

import asyncpraw
import nest_asyncio
import asyncio

# Allow nested event loops in Jupyter notebooks (including Google Colab)
nest_asyncio.apply()

# Create global variables to store user data and subreddit info
data = []
subreddits = {}

# Create a function to fetch user data asynchronously
async def fetch_user_data(username):
    async with asyncpraw.Reddit(
        client_id='gPWNZxE-hepzoXXwx2GcOQ',  # Replace with your actual Client ID
        client_secret='vrQVQDP3y5G-uogU7seq6wa-PgfFZQ',  # Replace with your actual client secret
        user_agent='MentalHealthAnalyzer/0.1',
    ) as reddit:
        user = await reddit.redditor(username)

        global data, subreddits  # Declare that we are using global variables

        user_data = []  # List to hold user comments
        subreddit_info = {}  # Dictionary to hold subreddit names and descriptions

        # Fetch comments from the user with a limit of 20
        async for comment in user.comments.new(limit=20):
            subreddit = comment.subreddit
            await subreddit.load()  # Load subreddit details to access all attributes

            # Add the comment details to user_data
            user_data.append(comment.body)  # Store only the comment body

            # Store subreddit info if not already stored
            if subreddit.display_name not in subreddit_info:
                subreddit_info[subreddit.display_name] = subreddit.public_description

        # Assign the fetched data to global variables
        data = user_data
        subreddits = subreddit_info

# Create a function to run the async code
async def main():
    username_input = input("Enter Reddit username: ")
    await fetch_user_data(username_input)

# Run the main function
await main()

import pandas as pd

# Create a DataFrame for comments
reddit_comments_df = pd.DataFrame(data, columns=['Comment'])

# Display Comments DataFrame
print("Comments DataFrame:")
print(reddit_comments_df)

# Create a DataFrame for only subreddit descriptions
subreddits_df = pd.DataFrame(list(subreddits.values()), columns=['Description'])

# Display Subreddits DataFrame
print("\nSubreddit Descriptions DataFrame:")
print(subreddits_df)

Enter Reddit username: cats
Comments DataFrame:
                                              Comment
0                                                Very
1   Take it as a $10k lesson and move on.\n\nI’d s...
2                                                Both
3   CRO has two chains actually. Crypto.com chain ...
4   [https://leaderboard.bitfinex.com/](https://le...
5   Everyone buys shit to sell later at a higher p...
6   People love fun. NFTs are fun. End of discussion.
7   It will hit 63k, 100k, and eventually 500k. Th...
8   How does burning a fee turn a currency into a ...
9   Scam. The actual website is lostpoets.xyz.\n\n...
10                       Nano has no smart contracts.
11  You will see old lockers under "Archives".\n\n...
12                                               USDC
13  PancakeSwap if you're willing to put liquidity...
14  SafeMoon killer is here guys. Invest or get le...
15  Better than GhostFace and HODL. Love the auto ...
16  Great team. Very active in cha

In [ ]:
# COLLECTING USER INTERACTION DATA FROM WHATSAPP (Userchats with most frequent)

In [4]:
import pandas as pd

# List of file paths (add each file path here)
file_paths = [
    "/content/chat.txt",
    "/content/chat2.txt",
]

try:
    # Initialize an empty list to store all messages
    all_messages = []

    # Loop through each file and process
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            messages = file.readlines()

        # Process each message to extract only the content after ': '
        for msg in messages:
            if ': ' in msg:
                content = msg.split(': ', 1)[1].strip()  # Get everything after the first ': '
                all_messages.append(content)

    # Create a DataFrame from the collected messages
    wasap_msg_df = pd.DataFrame(all_messages, columns=['msg'])

    print("First few rows of the combined DataFrame:")
    print(wasap_msg_df.head())
    print("\nTotal number of messages:", len(wasap_msg_df))

except FileNotFoundError as e:
    print(f"File not found: {e.filename}")
except Exception as e:
    print(f"An error occurred: {e}")

First few rows of the combined DataFrame:
                                                 msg
0                                    Hey… you awake?
1                Yeah, can't sleep again. What's up?
2  Just feeling... off, you know? Everything feel...
3  I get it. It’s been the same for me. Every day...
4  Exactly. It’s like I’m surrounded by people, b...

Total number of messages: 43


In [ ]:
# predicting using trained model

In [7]:
import pickle

# Load the model using pickle
with open('/content/model_nb.pkl', 'rb') as f:
    model = pickle.load(f)

# Define the prediction function
def predict_status(sentence):

    # Get prediction
    prediction = model.predict([sentence])

    # Apply threshold and return status
    predicted_class = 1 if prediction[0] > 0.5 else 0
    return 'NORMAL' if predicted_class == 1 else 'NOT NORMAL'

# Apply the prediction function to DataFrames
reddit_comments_df['res_status'] = reddit_comments_df['Comment'].apply(predict_status)
subreddits_df['res_status'] = subreddits_df['Description'].apply(predict_status)
wasap_msg_df['res_status'] = wasap_msg_df['msg'].apply(predict_status)

In [15]:
def get_overall_recommendation(dfs_dict):
    total_normal = 0
    total_not_normal = 0

    # ANSI escape codes for colors
    RED = "\033[91m"
    GREEN = "\033[92m"
    RESET = "\033[0m"  # To reset the color back to default

    # Calculate totals across all dataframes
    for df_name, df in dfs_dict.items():
        counts = df['res_status'].value_counts()
        total_normal += counts.get('NORMAL', 0)
        total_not_normal += counts.get('NOT NORMAL', 0)

    # Print the counts with color formatting
    print(f"\nTotal {GREEN}NORMAL{RESET} messages across all platforms: {GREEN}{total_normal}{RESET}")
    print(f"Total {RED}NOT NORMAL{RESET} messages across all platforms: {RED}{total_not_normal}{RESET}")

    # Check if the "NOT NORMAL" count is 40% or more of the "NORMAL" count
    if total_normal > 0 and (total_not_normal / total_normal) >= 0.4:
        print(f"\n{RED}WARNING: The person may be prone to mental health issues (NOT NORMAL messages are 40% or more of NORMAL messages).{RESET}")

    # Make final recommendation based on the total counts
    if total_normal > total_not_normal:
        print(f"\nSTAGE-1 : FINAL ASSESSMENT: {GREEN}No major mental health concerns - normal.{RESET}")
    else:
        print(f"\nSTAGE-1 : FINAL ASSESSMENT: {RED}Mental health concerns detected - doctor recommended.{RESET}")


In [16]:
# Create dictionary of dataframes
dfs = {
    'Reddit Comments': reddit_comments_df,
    'Subreddits': subreddits_df,
    'WhatsApp': wasap_msg_df
}

# Get overall recommendation with the new feature
get_overall_recommendation(dfs)


Total NORMAL messages across all platforms: 47
Total NOT NORMAL messages across all platforms: 21


STAGE-1 : FINAL ASSESSMENT: No major mental health concerns - normal.
